In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os

In [3]:
#import os
print(os.listdir("/gdrive/My Drive/YAZ20411/Fruits360"))

['test', 'training']


In [4]:
DATADIR = '/gdrive/My Drive/YAZ20411/Fruits360/training/'

CATEGORIES = ["Apple Golden 1","Avocado","Banana","Cherry 1","Cocos","Kiwi","Lemon","Mango","Orange"]

In [5]:
IMG_SIZE = 100

In [6]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [7]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  4306


In [8]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [11]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(3099, 100, 100, 3)
(862, 100, 100, 3)
(345, 100, 100, 3)
(3099,)
(862,)
(345,)


In [13]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=9)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=9)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=9)

In [15]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(64, (3,3), padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(9))
model.add(Activation('softmax'))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [16]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [17]:
history = model.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 50
                    )

Epoch 1/50
49/49 [==============================] - ETA: 0s - loss: 0.8931 - accuracy: 0.9058
Epoch 1: val_accuracy improved from -inf to 0.27536, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


49/49 [==============================] - 19s 180ms/step - loss: 0.8931 - accuracy: 0.9058 - val_loss: 48.1995 - val_accuracy: 0.2754
Epoch 2/50
49/49 [==============================] - ETA: 0s - loss: 0.0184 - accuracy: 0.9977
Epoch 2: val_accuracy improved from 0.27536 to 0.87826, saving model to model.h5
49/49 [==============================] - 7s 137ms/step - loss: 0.0184 - accuracy: 0.9977 - val_loss: 1.3515 - val_accuracy: 0.8783
Epoch 3/50
49/49 [==============================] - ETA: 0s - loss: 0.1652 - accuracy: 0.9910
Epoch 3: val_accuracy did not improve from 0.87826
49/49 [==============================] - 6s 116ms/step - loss: 0.1652 - accuracy: 0.9910 - val_loss: 680.3400 - val_accuracy: 0.1043
Epoch 4/50
49/49 [==============================] - ETA: 0s - loss: 0.6393 - accuracy: 0.9606
Epoch 4: val_accuracy did not improve from 0.87826
49/49 [==============================] - 6s 115ms/step - loss: 0.6393 - accuracy: 0.9606 - val_loss: 761.0435 - val_accuracy: 0.1304
Epoch

In [18]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

11/11 [==============================] - 1s 14ms/step - loss: 3.8009e-09 - accuracy: 1.0000
Validation Accuracy:  1.0
27/27 [==============================] - 1s 36ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Validation Accuracy:  1.0
97/97 [==============================] - 1s 15ms/step - loss: 9.1167e-09 - accuracy: 1.0000
Validation Accuracy:  1.0


In [ ]:
# RESNET50 RESNET50V2
# INCEPTION
# XCEPTION
# VGG16, VGG19
# MOBILENET V1, 2, 3
# DENSENET